In [140]:
import yfinance as yf
import pandas as pd

ftse = pd.read_html('https://en.wikipedia.org/wiki/FTSE_100_Index')[4]
ftse_tickers = ftse['Ticker']
ftse_tickers = ftse_tickers.to_list()
ftse_tickers = [ticker + '.L' for ticker in ftse_tickers]

snp = pd.read_html('https://en.wikipedia.org/wiki/S%26P_100')[2]
snp_tickers = snp['Symbol']
snp_tickers = snp_tickers.to_list()
snp_tickers[snp_tickers.index('BRK.B')] = 'BRK-B'

tickers = yf.Tickers(' '.join(ftse_tickers + snp_tickers))

ticker_df = pd.DataFrame(columns=['ticker', 'comp_name', 'exchange', 'sector', 'market_cap', 'currency'])
ticker_df.ticker = ftse_tickers + snp_tickers
ticker_df.comp_name = [ticker.info['shortName'] for ticker in tickers.tickers.values()]
ticker_df.comp_name = ticker_df.comp_name.str.replace("'", "''")
ticker_df.exchange = [ticker.info['exchange'] for ticker in tickers.tickers.values()]
ticker_df.market_cap = [ticker.info['marketCap'] for ticker in tickers.tickers.values()]
ticker_df.currency = [ticker.info['financialCurrency'] for ticker in tickers.tickers.values()]

sectors = []
for ticker in tickers.tickers.values():
    try:
        sectors.append(ticker.info['sector'])
    except KeyError:
        sectors.append(None)

ticker_df.sector = sectors
ticker_df['start_date'] = pd.to_datetime('today').date()

import psycopg as pg

with pg.connect(host='localhost', dbname='Stocks', user='postgres', password='420691', port='5432') as conn:
    with conn.cursor() as cur:
        for _, row in ticker_df.iterrows():
            cur.execute(f"INSERT INTO tickers (ticker, comp_name, exchange, sector, market_cap, currency, start_date) VALUES ('{row['ticker']}', '{row['comp_name']}', '{row['exchange']}', '{row['sector']}', {row['market_cap']}, '{row['currency']}', '{row['start_date']}');")
            
        conn.commit()

,ticker,comp_name,exchange,sector,market_cap,currency,start_date
0,III.L,3I GROUP PLC ORD 73 19/22P,LSE,None,34716143616,GBP,2024-12-27
1,ADM.L,ADMIRAL GROUP PLC ORD 0.1P,LSE,Financial Services,7759961088,GBP,2024-12-27
2,AAF.L,AIRTEL AFRICA PLC ORD USD0.50,LSE,Communication Services,4167548672,USD,2024-12-27
3,ALW.L,ALLIANCE WITAN PLC ORD 2.5P,LSE,None,4946373120,GBP,2024-12-27
4,AAL.L,ANGLO AMERICAN PLC ORD USD0.549,LSE,Basic Materials,28392769536,USD,2024-12-27
...,...,...,...,...,...,...,...
196,V,Visa Inc.,NYQ,Financial Services,618392322048,USD,2024-12-27
197,VZ,Verizon Communications Inc.,NYQ,Communication Services,168174714880,USD,2024-12-27
198,WFC,Wells Fargo & Company,NYQ,Financial Services,236926517248,USD,2024-12-27
199,WMT,Walmart Inc.,NYQ,Consumer Defensive,736281886720,USD,2024-12-27


In [64]:
us = yf.download(snp_tickers, start='2020-01-01')
us.count()

[*********************100%***********************]  101 of 101 completed


Price      Ticker
Adj Close  AAPL      1256
           ABBV      1256
           ABT       1256
           ACN       1256
           ADBE      1256
                     ... 
Volume     V         1256
           VZ        1256
           WFC       1256
           WMT       1256
           XOM       1256
Length: 606, dtype: int64

In [65]:
uk = yf.download(ftse_tickers, start='2020-01-01')
uk.count()

[*********************100%***********************]  100 of 100 completed


Price      Ticker
Adj Close  AAF.L     1259
           AAL.L     1258
           ABF.L     1260
           ADM.L     1260
           AHT.L     1260
                     ... 
Volume     UU.L      1260
           VOD.L     1257
           WEIR.L    1259
           WPP.L     1258
           WTB.L     1259
Length: 600, dtype: int64